In [1]:
def request_github_no_json(url,personal_access=personal_access_token):
    headers = {"Authorization": f"Bearer {personal_access}","Accept": "application/vnd.github.v3.star+json","content-type": "application/json"}

    res=requests.get(url ,headers=headers)
    return res

NameError: name 'personal_access_token' is not defined

In [ ]:
def lista_fecha_a_df(lista,nombre='Grafica temporal', plot=False):
  df=pd.DataFrame({nombre:lista})
  df[f'freq_{nombre}']=pd.to_datetime(df[nombre]).dt.date
  df1=df.groupby(f'freq_{nombre}').size().cumsum()
  if plot: df1.plot()
  return df1

In [ ]:
def create_stargazers_count(user,repo):
    """this function creates the stargazers count dataframe"""

    star_count_url = "https://api.github.com/repos/"+user+"/"+repo
    # response = requests.request("GET", star_count_url)
    response = request_github_no_json(star_count_url)
    total_star_count = response.json()['stargazers_count']
    loops = int(total_star_count / 100) + 1
    star_trends_url = "https://api.github.com/repos/"+user+"/"+repo+"/stargazers"
    star_trends_resp = []
    headers = {
    "Accept": "application/vnd.github.v3.star+json",
    "content-type": "application/json"
    }
    for page in range(loops):
        # response = requests.request("GET", star_trends_url+"?per_page=100"+"&page="+str(page+1), headers=headers).json()
        response = request_github_no_json(star_trends_url+"?per_page=100"+"&page="+str(page+1)).json()
        star_trends_resp.extend(response)

    df = json_normalize(star_trends_resp)

    #df['starred_date'] = pd.to_datetime(df['starred_at']).dt.date

    #star_trend_df = df.groupby(['starred_date'])['starred_date'].count().cumsum().reset_index(name="count")

    return df

In [ ]:
def create_repos_count(user):
    """crea el df temporal de repos"""

    repos_url = "https://api.github.com/users/"+user+"/repos"
    repos_resp = []
   
    page=0
    response = request_github_no_json(repos_url+"?per_page=100"+"&page="+str(page+1)).json() # primeras 100
    repos_resp.extend(response)
    while len(response)!=0:
        page+=1
        response = request_github_no_json(repos_url+"?per_page=100"+"&page="+str(page+1)).json()
        repos_resp.extend(response)
        
    df = json_normalize(repos_resp)

    return df

In [ ]:
def create_starred_count(user):
    """crea el df temporal de starred"""

    starred_url = "https://api.github.com/users/"+user+"/starred"
    repos_resp = []
   
    page=0
    response = request_github_no_json(starred_url+"?per_page=100"+"&page="+str(page+1)).json() # primeras 100
    repos_resp.extend(response)
    while len(response)!=0:
        page+=1
        response = request_github_no_json(starred_url+"?per_page=100"+"&page="+str(page+1)).json()
        repos_resp.extend(response)
        
    df = json_normalize(repos_resp)

    return df

In [ ]:
def create_forks_count(user, repo):
    """crea el df temporal de forks"""

    forks_url = f"https://api.github.com/repos/{user}/{repo}/forks"
    repos_resp = []
   
    page=0
    response = request_github_no_json(forks_url+"?per_page=100"+"&page="+str(page+1)).json() # primeras 100
    repos_resp.extend(response)
    while len(response)!=0:
        page+=1
        response = request_github_no_json(forks_url+"?per_page=100"+"&page="+str(page+1)).json()
        repos_resp.extend(response)
        
    df = json_normalize(repos_resp)

    return df

In [ ]:
def create_commits_count(user, repo):
    """crea el df temporal de commits"""

    commits_url = f"https://api.github.com/repos/{user}/{repo}/commits"
    repos_resp = []
   
    page=0
    response = request_github_no_json(commits_url+"?per_page=100"+"&page="+str(page+1)).json() # primeras 100
    repos_resp.extend([i['commit']['author']['date'] for i in response])
    while len(response)!=0:
        page+=1
        response = request_github_no_json(commits_url+"?per_page=100"+"&page="+str(page+1)).json()
        repos_resp.extend([i['commit']['author']['date'] for i in response])
        
    df = pd.DataFrame({'created_at':repos_resp})

    return df

In [ ]:
def crear_col_temporal_repo(fun,columna,user,repo,q=4,plot=False):
  '''
  fun:funcion como create_stargazers_count que regresa la cuenta de la serie de tiempo 
  '''
  try:
    cuentas_temporales=fun(user,repo) # Esto hace requests
    serie_acum=lista_fecha_a_df(cuentas_temporales[columna],'a',plot)
    cuantiles=pd.qcut(serie_acum, q=q, retbins=True)[1]
  except:
    return [np.nan]*4

  return [cuantiles[1]]+list(cuantiles[2:] / cuantiles[1:-1])


In [2]:
def obtener_hist_grafica(user,repo,atributo,nom_col,plot=True):

    consulta_url = f"https://api.github.com/repos/{user}/{repo}/{atributo}"
    repos_resp = []
    
    page=1
    response_no_json = request_github_no_json(consulta_url+"?per_page=100"+"&page="+str(page))
    response=response_no_json.json() # primeras 100
    
    # Estos try son para cuando excedes el límite de 5000 por hora 
    try:
        repos_resp.extend([re[nom_col] for re in response])
    except:
        print('Número máximo alcanzado, esperando 30s')
        time.sleep(30)
        page=page-1
        

    while len(response)!=0:
        page+=1
        response = request_github_no_json(consulta_url+"?per_page=100"+"&page="+str(page)).json()
        
        try:
            repos_resp.extend([re[nom_col] for re in response])
        except:
            print('Número máximo alcanzado, esperando 30s')
            time.sleep(30)
            page=page-1
            
            

  
    serie_acum=lista_fecha_a_df(repos_resp)
    try:
        cuantiles=pd.qcut(serie_acum, q=4, retbins=True)[1]
    
    except:
        return '0,0,0,0'

    if plot:
      fig=px.scatter(serie_acum,title=f'Histórico {atributo}')
      fig.add_hline(y=cuantiles[0])
      fig.add_hline(y=cuantiles[1])
      fig.add_hline(y=cuantiles[2])
      fig.add_hline(y=cuantiles[3])
      fig.show()
    return serie_acum, cuantiles

In [3]:
def obtener_info_repo_plot(user,repo):
    """Regresa json con info histórica y del repo"""

    star_count_url = "https://api.github.com/repos/"+user+"/"+repo

    response1 = request_github_no_json(star_count_url)
     
    serie_estrellas,cuantiles_estrellas = obtener_hist_grafica(user,repo,'stargazers','starred_at')
    serie_forks,cuantiles_forks = obtener_hist_grafica(user,repo,'forks','created_at')

    resp_json=response1.json()
    resp_json['cuant_estrellas']=cuantiles_estrellas
    resp_json['cuant_forks']=cuantiles_forks
 
    return resp_json